# Generating puzzles with multiple solutions
Puzzles with multiple solutions can create very interesting effects, such as the puzzle shown in the video "[How can a jigsaw have two distinct solutions?](https://www.youtube.com/watch?v=b5nElEbbnfU)" on the Stand-Up Maths Youtube channel where the two ways of solving the jigsaw puzzle either create an image of a coffee cup or a donut. This page describes how such puzzles can be generated by first picking a random scrambling of the puzzle pieces, and then generating a puzzle that is solvable in both its unscrambled and scrambled state. There is also Python code.

# Describing puzzles using vectors
In order for a jigsaw puzzle to be solvable in more than one way, there must be several connections between pieces that have the same shape. Otherwise, if every connection was unique, each side of a piece would only be able to connect with a single side of a single other piece, and there would only be a single solution to the puzzle. This means that each side of each puzzle piece can be grouped with other puzzle piece sides with a similar shape, and these groups can be given numbers.

It is useful to give the shapes numbers so that shapes that are compatible with each other have inverse numbers. For example, shape 1 is compatible with shape -1, shape 2 is compatible with shape -2, and so forth. This makes it easy to check whether two shapes are compatible: they are compatible if they add up to 0, and incompatible if they do not. I prefer to think of the positive numbers as all the shapes that stick out and the negative numbers as the indented shapes, but this is fully arbitrary. Finally, the outer sides of the pieces at the edge of the jigsaw puzzle are just shaped like straight lines, and do not connect to anything. These non-connecting shapes are labelled 0.

A puzzle can be described mathematically as a vector containing the shapes of each side of all the pieces of the puzzle. The order of the sides is also arbitrary, but here we order the sides in order left, top, right, bottom, starting with the top left piece and going row by row. The ordering of sides for a 2x2 puzzle is illustrated in the following figure.

<img src="sidenumbers.png" style="width:500px; margin-left:auto; margin-right:auto;"/>

Thus, if we assume that all the tabs in the image are in shape group 1 and all the indentations are in shape group -1, the vector describing this puzzle is:
$$
    \vec{p} = \begin{pmatrix}
        0 \\
        0 \\
        -1 \\
        1 \\
        1 \\
        0 \\
        0 \\
        1 \\
        0 \\
        -1 \\
        -1 \\
        0 \\
        1 \\
        -1 \\
        0 \\
        0
    \end{pmatrix}
$$

In order to solve a puzzle, the pieces have to be moved around and rotated. This can be done by performing matrix operations on the vector. For instance, the matrix that swaps the position of the top left and bottom right pieces looks like this, and then rotates the bottom right piece once counter-clockwise, looks like this:
$$
    \mathbf{T} = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    \end{pmatrix}
$$

# Validating if a puzzle is solved
A puzzle can be validated by checking each entry in the puzzle vector: if the entry corresponds to a side on the edge of the puzzle, its value must be zero. If an entry corresponds to a side which connects to another side in the puzzle, the sum of that entry and the other side's entry must be zero. This set of operations can be encoded into a verification matrix, $\mathbf{V}$. For example, the verification matrix for a 2x2 puzzle looks like this:
$$
    \mathbf{V} = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$
This matrix extracts only the entry itself for sides on the edge of the jigsaw puzzle, while it adds the value of both entries in the case of sides that face each other. Similar matrices can be constructed for any other dimension of jigsaw puzzle.

Using the verification matrix, puzzles can be validated by ensuring that they satisfy the following equation
$$
    \mathbf{V}\vec{p} = \vec{0}
$$

In [ ]:
%load verification_matrix.py

 # Finding puzzles with multiple solutions
 Any arbitrary scrambling (i.e. movement and/or rotation of the pieces) of a puzzle can be described using a transformation matrix, $\mathbf{T}$. If the matrix equation $\mathbf{V}\mathbf{T}\vec{p} = \vec{0}$ holds, then this scrambling is a solution to the puzzle. For a given transformation $\mathbf{T}$ and a given verification matrix $\mathbf{V}$, it is possible to construct a puzzle vector $\vec{p}$ such that the following system of matrix equations hold true:
 $$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
 $$
 $$
    \mathbf{V}\vec{p} = \vec{0}
 $$

A puzzle $\vec{p}$ that satisfies these constraints must have at least two solutions, since both its unscrambled state and its state that is scrambled according to $\mathbf{T}$ are solutions.

In [ ]:
%load puzzle_generator.py

# Random transformations and scramble similarity
Since we now know how to generate a puzzle that has at least two solutions given an arbitrary transformation, we can generate a random transformation and build a puzzle with multiple solutions using that. However, it is important to keep in mind that the transformation can't be completely arbitrary: only corner pieces can occupy the corners of the jigsaw puzzle, and only edge pieces can occupy the edges, and their orientation is determined by which edge they're on. All middle pieces can be arbitrarily rearranged and rotated.

In the video, Matt Parker also wants the two solutions to a puzzle to be as different as possible. One way of quantifying how similar the puzzle is after being scrambled is to count how many sides are adjacent to each other in the scrambled puzzle that were also adjacent in the original puzzle. This is easily countable: if every connecting shape in the original puzzle is unique, then every pair of sides that is still able to connect after scrambling the puzzle are sides that were connected in the original puzzle.

In [ ]:
%load transformations.py

# Counting the number of solutions
In order to achieve the effect in the video where the puzzle motif changes from a coffee cup to a donut, the puzzle should only have two solutions, otherwise it would be possible to put it back together in different ways that create nonsensical motifs. Since the above method guarantees that a puzzle has *at least* two solutions, we need to count the number of possible solutions to ensure that the puzzle has *only* two solutions.

Counting the number of possible solutions amounts to finding the number of matrices $\mathbf{T}$ that satisfies the equation
$$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
$$
where $\mathbf{V}$ and $\vec{p}$ are known. I haven't been able to prove this, but experience shows that $\mathbf{V}$ is always non-invertible, so I don't think there exists a better way of finding $\mathbf{T}$ than trial and error.

Since counting the number of solutions takes a long time, it is useful to weed out puzzles that obviously have more than two solutions. Any where two pieces have the exact same side shapes must have at least two solutions, since we can solve the puzzle by simply swapping those two pieces. Since we already know that our chosen puzzle also has a solution that is more complicated than just swapping two pieces, our puzzle must have more than two solutions

In [ ]:
%load solution_finder.py

The following cell contains code to draw the puzzle

In [ ]:
%load puzzle_drawer.py

In [ ]:
%load -r 8: main.py